# **Single REIT Snapshot**

## **Simulate Base Holding Data**

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

num_holdings = 10
weights = np.round(np.random.dirichlet(np.ones(num_holdings), size=1)[0], 2)

base_data = pd.DataFrame({
    'Holding': [f'REIT_{i+1}' for i in range(num_holdings)],
    'Weight': weights,
    'Price': np.round(np.random.uniform(30, 120, num_holdings), 2),
    'Shares Held': np.random.randint(100000, 500000, num_holdings),
    'EPS_now': np.round(np.random.uniform(1.5, 5.0, num_holdings), 2),
    'EPS_5y_ago': np.round(np.random.uniform(1.0, 3.5, num_holdings), 2),
    'Book Value/Share': np.round(np.random.uniform(25, 60, num_holdings), 2),
})

## **Calculate Per-Holding Values**

In [ ]:
# Derived holding metrics
base_data['Market Value'] = base_data['Price'] * base_data['Shares Held']
base_data['P/E'] = base_data['Price'] / base_data['EPS_now']
base_data['P/B'] = base_data['Price'] / base_data['Book Value/Share']
base_data['ROE'] = base_data['EPS_now'] / base_data['Book Value/Share']
base_data['Earnings Growth Rate'] = ((base_data['EPS_now'] / base_data['EPS_5y_ago'])**(1/5)) - 1

## **Compute Fund-Level Metrics**

In [ ]:
# Weighted averages
pe_ratio = np.sum(base_data['P/E'] * base_data['Weight'])
pb_ratio = np.sum(base_data['P/B'] * base_data['Weight'])
roe = np.sum(base_data['ROE'] * base_data['Weight'])
growth_rate = np.sum(base_data['Earnings Growth Rate'] * base_data['Weight'])
num_stocks = base_data['Holding'].nunique()
median_market_cap = np.median(base_data['Market Value']) / 1e9  # Convert to billions

## **Generate Risk/Return Metrics**

In [ ]:
# Simulate fund NAV over 3 years (monthly)
dates = pd.date_range(start='2022-01-01', periods=36, freq='ME')
nav = 100 + np.cumsum(np.random.normal(loc=0.5, scale=2, size=36))  # synthetic NAV
benchmark = 100 + np.cumsum(np.random.normal(loc=0.4, scale=1.8, size=36))  # synthetic MSCI returns

monthly_returns = pd.Series(nav).pct_change().dropna()
benchmark_returns = pd.Series(benchmark).pct_change().dropna()

# Alpha (CAPM-like), Beta, Std Dev
rf = 0.0125 / 12  # 3-month T-bill (approx. 1.25% annualized)
rm = benchmark_returns.mean()
rp = monthly_returns.mean()
beta = np.cov(monthly_returns, benchmark_returns)[0][1] / np.var(benchmark_returns)
alpha = rp - (rf + beta * (rm - rf))
std_dev = np.std(monthly_returns) * np.sqrt(12)

## **Static Fund Info**

In [ ]:
benchmark_name = "MSCI US Investable Market Real Estate 25/50 Index"
expense_ratio = round(np.random.uniform(0.6, 1.1), 2)
div_schedule = "Quarterly"
etf_assets = base_data['Market Value'].sum() / 1e6  # millions
fund_assets = etf_assets * 0.95  # simulate slightly less due to fees/liabilities
inception_date = "2004-09-23"

## **Final Output**

In [ ]:
summary = pd.DataFrame({
    'Metric': [
        'Number of Stocks', 'Median Market Cap ($B)', 'P/E Ratio', 'P/B Ratio',
        'ROE (%)', 'Earnings Growth Rate (%)', 'Alpha', 'Beta',
        'Standard Deviation (%)', 'Benchmark', 'Expense Ratio',
        'Dividend Schedule', 'ETF Net Assets ($M)', 'Fund Net Assets ($M)', 'Inception Date'
    ],
    'Value': [
        num_stocks, round(median_market_cap, 2), round(pe_ratio, 2), round(pb_ratio, 2),
        round(roe * 100, 2), round(growth_rate * 100, 2), round(alpha, 4),
        round(beta, 2), round(std_dev * 100, 2), benchmark_name, expense_ratio,
        div_schedule, round(etf_assets, 2), round(fund_assets, 2), inception_date
    ]
})

# **Single REIT Performance**

## **Generating output Excel Data - Single Portfolio**

In [ ]:
# 📦 Install dependencies (optional in Colab)
!pip install pandas numpy xlsxwriter

import pandas as pd
import numpy as np
from datetime import datetime

# 🎲 Seed for reproducibility
np.random.seed(42)

# STEP 1: Simulate 10 synthetic REIT holdings
num_holdings = 10
weights = np.round(np.random.dirichlet(np.ones(num_holdings), size=1)[0], 2)
weights[-1] = 1.0 - weights[:-1].sum()  # Make sure weights sum to 1

base_data = pd.DataFrame({
    'Holding': [f'REIT_{i+1}' for i in range(num_holdings)],
    'Weight': weights,
    'Price': np.round(np.random.uniform(30, 120, num_holdings), 2),
    'Shares Held': np.random.randint(100_000, 500_000, num_holdings),
    'EPS_now': np.round(np.random.uniform(1.5, 5.0, num_holdings), 2),
    'EPS_5y_ago': np.round(np.random.uniform(1.0, 3.5, num_holdings), 2),
    'Book Value/Share': np.round(np.random.uniform(25, 60, num_holdings), 2),
})

# STEP 2: Derived calculations
base_data['Market Value'] = base_data['Price'] * base_data['Shares Held']
base_data['P/E'] = base_data['Price'] / base_data['EPS_now']
base_data['P/B'] = base_data['Price'] / base_data['Book Value/Share']
base_data['ROE'] = base_data['EPS_now'] / base_data['Book Value/Share']
base_data['Earnings Growth Rate'] = ((base_data['EPS_now'] / base_data['EPS_5y_ago'])**(1/5)) - 1

# STEP 3: Fund-level aggregates
pe_ratio = np.sum(base_data['P/E'] * base_data['Weight'])
pb_ratio = np.sum(base_data['P/B'] * base_data['Weight'])
roe = np.sum(base_data['ROE'] * base_data['Weight'])
growth_rate = np.sum(base_data['Earnings Growth Rate'] * base_data['Weight'])
num_stocks = base_data['Holding'].nunique()
median_market_cap = np.median(base_data['Market Value']) / 1e9  # Convert to billions

# STEP 4: Simulate 3-year NAV & Benchmark
dates = pd.date_range(start='2022-01-01', periods=36, freq='M')
nav = 100 + np.cumsum(np.random.normal(loc=0.5, scale=2, size=36))
benchmark = 100 + np.cumsum(np.random.normal(loc=0.4, scale=1.8, size=36))

monthly_returns = pd.Series(nav).pct_change().dropna()
benchmark_returns = pd.Series(benchmark).pct_change().dropna()

# Risk metrics
rf = 0.0125 / 12  # Monthly risk-free rate
rm = benchmark_returns.mean()
rp = monthly_returns.mean()
beta = np.cov(monthly_returns, benchmark_returns)[0][1] / np.var(benchmark_returns)
alpha = rp - (rf + beta * (rm - rf))
std_dev = np.std(monthly_returns) * np.sqrt(12)

# STEP 5: Static fund info
benchmark_name = "MSCI US Investable Market Real Estate 25/50 Index"
expense_ratio = round(np.random.uniform(0.6, 1.1), 2)
div_schedule = "Quarterly"
etf_assets = base_data['Market Value'].sum() / 1e6
fund_assets = etf_assets * 0.95
inception_date = "2004-09-23"

# STEP 6: Output to summary DataFrame
summary = pd.DataFrame({
    'Metric': [
        'Number of Stocks', 'Median Market Cap ($B)', 'P/E Ratio', 'P/B Ratio',
        'ROE (%)', 'Earnings Growth Rate (%)', 'Alpha', 'Beta',
        'Standard Deviation (%)', 'Benchmark', 'Expense Ratio',
        'Dividend Schedule', 'ETF Net Assets ($M)', 'Fund Net Assets ($M)', 'Inception Date'
    ],
    'Value': [
        num_stocks, round(median_market_cap, 2), round(pe_ratio, 2), round(pb_ratio, 2),
        round(roe * 100, 2), round(growth_rate * 100, 2), round(alpha, 4),
        round(beta, 2), round(std_dev * 100, 2), benchmark_name, expense_ratio,
        div_schedule, round(etf_assets, 2), round(fund_assets, 2), inception_date
    ]
})

# STEP 7: Save to Excel
output_filename = "Synthetic_REIT_Summary.xlsx"
with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
    base_data.to_excel(writer, sheet_name="Holdings Detail", index=False)
    summary.to_excel(writer, sheet_name="Fund Summary", index=False)
    pd.DataFrame({'Date': dates, 'NAV': nav, 'Benchmark': benchmark}).to_excel(writer, sheet_name="NAV and Benchmark", index=False)

print(f"✅ Synthetic REIT data saved as: {output_filename}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.3 MB/s eta 0:00:00
✅ Synthetic REIT data saved as: Synthetic_REIT_Summary.xlsx


/tmp/ipython-input-7-3314027069.py:42: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2022-01-01', periods=36, freq='M')


## **Generating output Excel Data - Multi Portfolio**

In [ ]:
# Step 1: Install dependencies (if running in Colab)

import pandas as pd
import numpy as np
from datetime import datetime

# Constants
np.random.seed(42)
account_codes = [f"ACCT10{i+1}" for i in range(5)]
account_names = [f"REIT Fund {chr(65+i)}" for i in range(5)]
years = pd.date_range(start="1995-12-31", periods=30, freq="YE")

# Step 2: Generate synthetic time-series data for each account
records = []

for code, name in zip(account_codes, account_names):
    base_pe = np.random.uniform(28, 42)
    base_pb = np.random.uniform(1.7, 2.4)
    base_roe = np.random.uniform(5, 10)
    base_growth = np.random.uniform(5, 12)
    base_etf_assets = np.random.uniform(200, 800)
    base_fund_assets = base_etf_assets * 0.95
    nav = 100

    for i, year in enumerate(years):
        pe = round(base_pe * (1 + np.random.normal(0.01, 0.03)), 2)
        pb = round(base_pb * (1 + np.random.normal(0.01, 0.03)), 2)
        roe = round(base_roe * (1 + np.random.normal(0.01, 0.03)), 2)
        growth = round(base_growth * (1 + np.random.normal(0.01, 0.05)), 2)
        etf_assets = round(base_etf_assets * (1 + 0.04) ** i + np.random.normal(0, 10), 2)
        fund_assets = round(etf_assets * 0.95, 2)
        return_pct = round(np.random.normal(7, 3), 2)
        nav = round(nav * (1 + return_pct / 100), 2)

        records.append({
            "Account Code": code,
            "Account Name": name,
            "Year": year.year,
            "P/E Ratio": pe,
            "P/B Ratio": pb,
            "ROE (%)": roe,
            "Earnings Growth Rate (%)": growth,
            "ETF Net Assets ($M)": etf_assets,
            "Fund Net Assets ($M)": fund_assets,
            "Return (%)": return_pct,
            "NAV": nav
        })

# Step 3: Convert to a DataFrame
long_df = pd.DataFrame(records)

# Step 4: Save to Excel
output_filename = "Synthetic_REIT_Timeseries_30Y.xlsx"
with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
    long_df.to_excel(writer, sheet_name="Fund Time Series", index=False)

print(f"✅ Synthetic REIT 30-year time series saved as: {output_filename}")


✅ Synthetic REIT 30-year time series saved as: Synthetic_REIT_Timeseries_30Y.xlsx


# **Comprehensive Multi-REIT**

## **Combining the two approaches**

In [ ]:
# Step 0: Install required packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)

# Setup
account_codes = [f"ACCT10{i+1}" for i in range(5)]
account_names = [f"REIT Fund {chr(65+i)}" for i in range(5)]
start_date = datetime(1995, 1, 1)
end_date = datetime(2024, 12, 31)
dates_daily = pd.date_range(start=start_date, end=end_date, freq='D')
years = pd.date_range(start="1995-12-31", periods=30, freq="Y")

# Containers
all_holdings, all_fund_summary, all_nav_benchmark = [], [], []
all_sector_allocations, all_contributions, all_disclosures = [], [], []

# Predefined sectors
sectors = ['Industrial', 'Retail', 'Residential', 'Office', 'Data Centers']

# Generate for each account
for code, name in zip(account_codes, account_names):
    ### 1. HOLDINGS
    num_holdings = 10
    weights = np.round(np.random.dirichlet(np.ones(num_holdings), size=1)[0], 2)
    weights[-1] = 1.0 - weights[:-1].sum()

    holdings = pd.DataFrame({
        'Account Code': code,
        'Account Name': name,
        'Holding': [f'{name} - REIT_{i+1}' for i in range(num_holdings)],
        'Weight': weights,
        'Sector': np.random.choice(sectors, num_holdings),
        'Price': np.round(np.random.uniform(30, 120, num_holdings), 2),
        'Shares Held': np.random.randint(100_000, 500_000, num_holdings),
        'EPS_now': np.round(np.random.uniform(1.5, 5.0, num_holdings), 2),
        'EPS_5y_ago': np.round(np.random.uniform(1.0, 3.5, num_holdings), 2),
        'Book Value/Share': np.round(np.random.uniform(25, 60, num_holdings), 2),
    })

    holdings['Market Value'] = holdings['Price'] * holdings['Shares Held']
    holdings['P/E'] = holdings['Price'] / holdings['EPS_now']
    holdings['P/B'] = holdings['Price'] / holdings['Book Value/Share']
    holdings['ROE'] = holdings['EPS_now'] / holdings['Book Value/Share']
    holdings['Earnings Growth Rate'] = ((holdings['EPS_now'] / holdings['EPS_5y_ago'])**(1/5)) - 1

    all_holdings.append(holdings)

    ### 2. FUND SUMMARY (Annual)
    fund_summary_rows = []
    base_pe = np.average(holdings['P/E'], weights=holdings['Weight'])
    base_pb = np.average(holdings['P/B'], weights=holdings['Weight'])
    base_roe = np.average(holdings['ROE'], weights=holdings['Weight'])
    base_growth = np.average(holdings['Earnings Growth Rate'], weights=holdings['Weight']) * 100
    base_etf_assets = holdings['Market Value'].sum() / 1e6
    nav = 100

    for i, year in enumerate(years):
        pe = round(base_pe * (1 + np.random.normal(0.01, 0.03)), 2)
        pb = round(base_pb * (1 + np.random.normal(0.01, 0.03)), 2)
        roe = round(base_roe * (1 + np.random.normal(0.01, 0.03)) * 100, 2)
        growth = round(base_growth * (1 + np.random.normal(0.01, 0.05)), 2)
        etf_assets = round(base_etf_assets * (1 + 0.04) ** i + np.random.normal(0, 10), 2)
        fund_assets = round(etf_assets * 0.95, 2)
        return_pct = round(np.random.normal(7, 3), 2)
        nav = round(nav * (1 + return_pct / 100), 2)

        fund_summary_rows.append({
            "Account Code": code,
            "Account Name": name,
            "Year": year.year,
            "P/E Ratio": pe,
            "P/B Ratio": pb,
            "ROE (%)": roe,
            "Earnings Growth Rate (%)": growth,
            "ETF Net Assets ($M)": etf_assets,
            "Fund Net Assets ($M)": fund_assets,
            "Return (%)": return_pct,
            "NAV": nav
        })

    all_fund_summary.append(pd.DataFrame(fund_summary_rows))

    ### 3. DAILY NAV & BENCHMARK
    nav_vals = 100 + np.cumsum(np.random.normal(loc=0.03, scale=0.4, size=len(dates_daily)))
    bench_vals = 100 + np.cumsum(np.random.normal(loc=0.025, scale=0.3, size=len(dates_daily)))

    nav_df = pd.DataFrame({
        'Account Code': code,
        'Account Name': name,
        'Date': dates_daily,
        'NAV': nav_vals,
        'Benchmark': bench_vals
    })
    all_nav_benchmark.append(nav_df)

    ### 4. SECTOR ALLOCATION
    sector_df = holdings.groupby('Sector')['Weight'].sum().reset_index()
    sector_df['Account Code'] = code
    sector_df['Account Name'] = name
    all_sector_allocations.append(sector_df)

    ### 5. CONTRIBUTION TO RETURN
    contrib_df = holdings[['Account Code', 'Account Name', 'Holding', 'Weight']].copy()
    contrib_df['Contribution to Return (%)'] = np.round(contrib_df['Weight'] * np.random.normal(7, 2), 2)
    all_contributions.append(contrib_df)

    ### 6. DISCLOSURE/QUALITATIVE
    disclosures = {
        "Account Code": code,
        "Account Name": name,
        "Strategy Description": f"{name} focuses on income-producing REITs across diversified property sectors.",
        "Investment Philosophy": "Long-term capital appreciation with disciplined risk control.",
        "Manager Commentary": f"In {years[-1].year}, the fund outperformed its benchmark due to strong performance in industrial and data center sectors.",
        "Disclosures": "Past performance does not guarantee future results. Synthetic data used for academic purposes only."
    }
    all_disclosures.append(disclosures)

# Combine and write to Excel
output_path = "Synthetic_REIT_Portfolios_30Y_FULL.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    pd.concat(all_holdings).to_excel(writer, sheet_name="Top Holdings", index=False)
    pd.concat(all_fund_summary).to_excel(writer, sheet_name="Fund Summary (Annual)", index=False)
    pd.concat(all_nav_benchmark).to_excel(writer, sheet_name="Daily NAV & Benchmark", index=False)
    pd.concat(all_sector_allocations).to_excel(writer, sheet_name="Sector Breakdown", index=False)
    pd.concat(all_contributions).to_excel(writer, sheet_name="Contribution to Return", index=False)
    pd.DataFrame(all_disclosures).to_excel(writer, sheet_name="Qualitative Content", index=False)

print(f"✅ Excel workbook created: {output_path}")


/tmp/ipython-input-9-4193098178.py:14: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  years = pd.date_range(start="1995-12-31", periods=30, freq="Y")


✅ Excel workbook created: Synthetic_REIT_Portfolios_30Y_FULL.xlsx


# **Daily Performance Metrics**

In [ ]:
import pandas as pd
import numpy as np

# 🎲 Step 1: Set seed for reproducibility
np.random.seed(42)

# 📆 Step 2: Create daily date range (2004–2024)
daily_dates = pd.date_range(start='2004-01-01', end='2024-12-31', freq='D')
num_days = len(daily_dates)

# 🧾 Step 3: Define 5 synthetic accounts
accounts = [
    {"Account Code": "ACCT101", "Account Name": "REIT Fund A"},
    {"Account Code": "ACCT102", "Account Name": "REIT Fund B"},
    {"Account Code": "ACCT103", "Account Name": "REIT Fund C"},
    {"Account Code": "ACCT104", "Account Name": "REIT Fund D"},
    {"Account Code": "ACCT105", "Account Name": "REIT Fund E"}
]

# 📦 Step 4: Simulate data for each account
all_data = []

for account in accounts:
    acct_code = account["Account Code"]
    acct_name = account["Account Name"]

    # NAV simulation (random walk)
    nav_returns = np.random.normal(loc=0.0003, scale=0.01, size=num_days)
    nav = [100]
    for r in nav_returns:
        nav.append(nav[-1] * (1 + r))
    nav = nav[1:]

    # Benchmark simulation
    benchmark_returns = np.random.normal(loc=0.00025, scale=0.009, size=num_days)
    benchmark = [100]
    for r in benchmark_returns:
        benchmark.append(benchmark[-1] * (1 + r))
    benchmark = benchmark[1:]

    # Dividend reinvestment multiplier (daily compounding)
    dividend_multiplier = np.random.uniform(0.00002, 0.00012, size=num_days)
    dividend_growth = np.cumprod(1 + dividend_multiplier)

    # Calculate performance metrics
    price_growth = np.array(nav[1:]) / np.array(nav[:-1])
    nav_return = price_growth - 1
    total_return = (price_growth * dividend_growth[:-1]) - 1
    benchmark_return = np.array(benchmark[1:]) / np.array(benchmark[:-1]) - 1

    # Build performance DataFrame
    df = pd.DataFrame({
        "Date": daily_dates[1:],  # drop first to match return lengths
        "Account Code": acct_code,
        "Account Name": acct_name,
        "NAV": nav[1:],  # same, drop first
        "NAV Return": nav_return,
        "Dividend Multiplier": dividend_growth[:-1],
        "Total Return": total_return,
        "Benchmark Return": benchmark_return,
        "PerformanceType": "Portfolio Net",
        "PerformanceFrequency": "D"
    })

    all_data.append(df)

# 🔗 Step 5: Combine all accounts into one dataset
performance_df_all = pd.concat(all_data, ignore_index=True)

# 💾 Step 6: Save to Excel
performance_df_all.to_excel("Synthetic_Daily_Performance.xlsx", index=False)

print("✅ File saved: Synthetic_Performance_With_Account_Names.xlsx")



✅ File saved: Synthetic_Performance_With_Account_Names.xlsx


In [ ]:
from google.colab import files
files.download("Synthetic_Daily_Performance.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>